In [64]:
%matplotlib inline
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.signal import savgol_filter
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc
from scipy.stats import linregress
from scipy import stats
from typing import Sequence, Tuple
from typing import Dict, Tuple
# ─── I/O Helpers ────────────────────────────────────────────────────────────

def get_data_from_mat_for_calcu(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:]
    return var.T

def get_data_from_nc(file, variable):
    file_obj = nc.Dataset(file)
    data = file_obj.variables[variable]
    var_data = np.array(data)
    var_data = var_data[:, 29:, :]
    var_data[var_data > 1000000] = np.nan
    var_data = np.squeeze(var_data)
    return var_data

def get_data_from_mat(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:] 
    return var

str_base = '' #TODO 
ar6_region = get_data_from_mat_for_calcu(str_base + '/plotting_tools/ar6_region.mat','ar6_region')
AREA = get_data_from_mat(str_base + '/plotting_tools/AREA.mat', 'AREA') # Area for CLM_data
irr_diff = get_data_from_mat_for_calcu(str_base + '/plotting_tools/irr_diff_out.mat', 'irr_diff_out')

# ─── Model‐specific loader ──────────────────────────────────────────────────
# Here we have a function for each ESM to read the variable
# This is because the folders and the names are all model-specific
def get_data_cesm2(variable):
    
    str_start = '/water_fluxes/CESM2/CESM2_'
    str_mid = '_1901_2014_'
    str_end = '_yearmean'
    
    data_irr01 = get_data_from_nc(str_base + str_start + 'IRR01' + str_mid + variable + str_end, variable)
    data_noi01 = get_data_from_nc(str_base + str_start + 'NOI01' + str_mid + variable + str_end, variable)

    data_irr02 = get_data_from_nc(str_base + str_start + 'IRR02' + str_mid + variable + str_end, variable)
    data_noi02 = get_data_from_nc(str_base + str_start + 'NOI02' + str_mid + variable + str_end, variable)

    data_irr03 = get_data_from_nc(str_base + str_start + 'IRR03' + str_mid + variable + str_end, variable)
    data_noi03 = get_data_from_nc(str_base + str_start + 'NOI03' + str_mid + variable + str_end, variable)

    data_irr = (data_irr01 + data_irr02 + data_irr03) / 3 * 365 * 86400
    data_noi = (data_noi01 + data_noi02 + data_noi03) / 3 * 365 * 86400
    
    return data_irr[:-1,:,:], data_noi[:-1,:,:]

def get_data_e3sm(variable3):
    
    str_start = '/water_fluxes/E3SMv2/E3SM_'
    str_mid = '_1901_2014_'
    str_end = '_yearmean_0.9x1.25'
    
    data_irr01 = get_data_from_nc(str_base + 'IRR01' + str_mid + variable3 + str_end, variable3)
    data_noi01 = get_data_from_nc(str_base + 'NOI01' + str_mid + variable3 + str_end, variable3)

    data_irr02 = get_data_from_nc(str_base + 'IRR02' + str_mid + variable3 + str_end, variable3)
    data_noi02 = get_data_from_nc(str_base + 'NOI02' + str_mid + variable3 + str_end, variable3)

    data_irr = (data_irr01 + data_irr02) / 2 * 365 * 86400
    data_noi = (data_noi01 + data_noi02) / 2 * 365 * 86400

    return data_irr[6:-1,:,:], data_noi[6:-1,:,:]

def get_data_cesm2_gw(variable):
    
    str_start = '/water_fluxes/CESM2_gw/CESM2_gw_'
    str_mid = '_1901_2014_'
    str_end = '_yearmean'
    
    data_irr01 = get_data_from_nc(str_base + str_start + 'IRR01' + str_mid + variable + str_end, variable)
    data_noi01 = get_data_from_nc(str_base + str_start + 'NOI01' + str_mid + variable + str_end, variable)

    data_irr02 = get_data_from_nc(str_base + str_start + 'IRR02' + str_mid + variable + str_end, variable)
    data_noi02 = get_data_from_nc(str_base + str_start + 'NOI02' + str_mid + variable + str_end, variable)

    data_irr03 = get_data_from_nc(str_base + str_start + 'IRR03' + str_mid + variable + str_end, variable)
    data_noi03 = get_data_from_nc(str_base + str_start + 'NOI03' + str_mid + variable + str_end, variable)

    data_irr = (data_irr01 + data_irr02 + data_irr03) / 3 * 365 * 86400
    data_noi = (data_noi01 + data_noi02 + data_noi03) / 3 * 365 * 86400
    
    return data_irr[:-1,:,:], data_noi[:-1,:,:]
    
def get_data_noresm(variable):
    
    str_start = '/water_fluxes/NorESM2/NorESM_'
    str_mid = '_1901_2014_'
    str_end = '_yearmean'
    
    data_irr01 = get_data_from_nc(str_base + str_start + 'IRR01' + str_mid + variable + str_end, variable)
    data_noi01 = get_data_from_nc(str_base + str_start + 'NOI01' + str_mid + variable + str_end, variable)

    data_irr02 = get_data_from_nc(str_base + str_start + 'IRR02' + str_mid + variable + str_end, variable)
    data_noi02 = get_data_from_nc(str_base + str_start + 'NOI02' + str_mid + variable + str_end, variable)

    data_irr03 = get_data_from_nc(str_base + str_start + 'IRR03' + str_mid + variable + str_end, variable)
    data_noi03 = get_data_from_nc(str_base + str_start + 'NOI03' + str_mid + variable + str_end, variable)

    data_irr01_temp = np.zeros([116,163,288])# the first year of ensemble 01 is missing
    data_irr01_temp[1:, :, :] = data_irr01
    data_irr01_temp[0, :, :] = (data_irr02[0, :, :] + data_irr03[0, :, :])/2
    data_irr01 = data_irr01_temp
    
    data_irr = (data_irr01 + data_irr02 + data_irr03) / 3 * 365 * 86400
    data_noi = (data_noi01 + data_noi02 + data_noi03) / 3 * 365 * 86400
    
    return data_irr[1:-1,:,:], data_noi[1:-1,:,:]
    
    
    
def get_data_ipsl(variable2):
    
    str_start = '/water_fluxes/IPSL-CM6/'
    str_end = '_1901_2014_Month.nc_yearmean_0.9x1.25'
    
    data_irr = get_data_from_nc(str_base + str_start + 'IRR01_'+ variable2 + str_end, variable2)
    data_noi = get_data_from_nc(str_base + str_start + 'NOI01_'+ variable2 + str_end, variable2)


    data_irr = data_irr * 365 * 86400
    data_noi = data_noi * 365 * 86400

    return data_irr, data_noi

def get_data_cnrm(variable2):
    
    str_start = '/water_fluxes/CNRM-CM6-1/'
    
    str_end = '.nc_yearmean_yearmean_0.9x1.25'
    
    data_irr01 = get_data_from_nc(str_start + variable2 + '_IRR01' + str_end, variable2)
    data_noi01 = get_data_from_nc(str_start + variable2 + '_NOI01' + str_end, variable2)

    data_irr02 = get_data_from_nc(str_start + variable2 + '_IRR02' + str_end, variable2)
    data_noi02 = get_data_from_nc(str_start + variable2 + '_NOI02' + str_end, variable2)

    data_irr03 = get_data_from_nc(str_start + variable2 + '_IRR03' + str_end, variable2)
    data_noi03 = get_data_from_nc(str_start + variable2 + '_NOI03' + str_end, variable2)

    data_irr04 = get_data_from_nc(str_start + variable2 + '_IRR04' + str_end, variable2)
    data_noi04 = get_data_from_nc(str_start + variable2 + '_NOI04' + str_end, variable2)

    data_irr05 = get_data_from_nc(str_start + variable2 + '_IRR05' + str_end, variable2)
    data_noi05 = get_data_from_nc(str_start + variable2 + '_NOI05' + str_end, variable2)

 
    
    data_irr = (data_irr01+data_irr02+data_irr03+data_irr04+data_irr05) / 5 * 365 * 86400
    data_noi = (data_noi01+data_noi02+data_noi03+data_noi04+data_noi05) / 5 * 365 * 86400

    return data_irr, data_noi
    
    
def get_data_miroc(variable):
    
    str_start = '/water_fluxes/MIROC-INTEG-ES/'
    
    str_mid = '_mon_MIROC_'
    str_end = '_1901-2014.nc_0.9x1.25_yearmean'
    
    data_irr01 = get_data_from_nc(str_start + 'tranirr-01' + variable + str_mid + 'IRR01' + str_end, variable)
    data_noi01 = get_data_from_nc(str_start + '1901irr-01' + variable + str_mid + 'NOI01' + str_end, variable)

    data_irr02 = get_data_from_nc(str_start + 'tranirr-02' + variable + str_mid + 'IRR02' + str_end, variable)
    data_noi02 = get_data_from_nc(str_start + '1901irr-02' + variable + str_mid + 'NOI02' + str_end, variable)

    data_irr03 = get_data_from_nc(str_start + 'tranirr-03' + variable + str_mid + 'IRR03' + str_end, variable)
    data_noi03 = get_data_from_nc(str_start + '1901irr-03' + variable + str_mid + 'NOI03' + str_end, variable)

    data_irr = (data_irr01 + data_irr02 + data_irr03) / 3 * 365 * 86400
    data_noi = (data_noi01 + data_noi02 + data_noi03) / 3 * 365 * 86400
    
    return data_irr, data_noi

In [2]:
MODEL_LOADERS = {
    'CESM2':    get_data_cesm2,
    'CESM2_gw': get_data_cesm2_gw,
    'NorESM':   get_data_noresm,
    'E3SM':     get_data_e3sm,
    'IPSL':     get_data_ipsl,
    'CNRM':     get_data_cnrm,
    'MIROC':    get_data_miroc,
}


# specify per-model which variable names to fetch
# tuple of (rain_var, snow_var) means load both and sum
# single-member tuple means loader returns P directly
PREC_VARS = {
    'CESM2':    ('RAIN_FROM_ATM',  'SNOW_FROM_ATM'),
    'CESM2_gw': ('RAIN_FROM_ATM',  'SNOW_FROM_ATM'),
    'NorESM':   ('RAIN_FROM_ATM',  'SNOW_FROM_ATM'),
    'E3SM':     ('RAIN',           'SNOW'),
    'IPSL':     ('pr',),    # loader returns P already
    'CNRM':     ('pr',),
    'MIROC':    ('pr',),
}

# container for all precipitation datasets
precip = {}

for model, vars_ in PREC_VARS.items():
    loader = MODEL_LOADERS[model]
    # if we have separate rain & snow variables, fetch both & sum
    if len(vars_) == 2:
        rain_var, snow_var = vars_
        irr_r = loader(rain_var)[0]  # [0] = IRR
        noi_r = loader(rain_var)[1]  # [1] = NOI
        irr_s = loader(snow_var)[0]
        noi_s = loader(snow_var)[1]

        precip[model] = {
            'IRR': irr_r + irr_s,
            'NOI': noi_r + noi_s
        }
    else:
        # single var: loader returns precipitation directly
        (var,) = vars_
        irr_p, noi_p = loader(var)
        precip[model] = {
            'IRR': irr_p,
            'NOI': noi_p
        }

# Example access:
# IWW_IRR_1901_2014_CESM2_P   = precip['CESM2']['IRR']
# IWW_NOI_1901_2014_CESM2_P   = precip['CESM2']['NOI']
# IWW_IRR_1901_2014_E3SM_P    = precip['E3SM']['IRR']
# IWW_NOI_1901_2014_MIROC_P   = precip['MIROC']['NOI']

RUNOFF_VARS = {
    'CESM2':    'QRUNOFF',
    'CESM2_gw': 'QRUNOFF',
    'NorESM':   'QRUNOFF',
    'E3SM':     'QRUNOFF',
    'IPSL':     'mrro',
    'CNRM':     'mrro',
    'MIROC':    'mrro',
}

# container for all runoff datasets
runoff = {}

for model, var in RUNOFF_VARS.items():
    loader = MODEL_LOADERS[model]
    irr_r, noi_r = loader(var)
    runoff[model] = {
        'IRR': irr_r,
        'NOI': noi_r
    }
    

ET_COMPONENTS = {
    'CESM2':    ['QFLX_EVAP_TOT'],
    'CESM2_gw': ['QFLX_EVAP_TOT'],
    'NorESM':   ['QFLX_EVAP_TOT'],
    'E3SM':     ['QSOIL', 'QVEGE', 'QVEGT'],
    'IPSL':     ['evspsbl'],
    'CNRM':     ['evspsbl'],
    'MIROC':    ['evspsbl', 'tran'],
}

evapotran = {}

for model, comps in ET_COMPONENTS.items():
    loader = MODEL_LOADERS[model]
    irr_parts = []
    noi_parts = []

    # load each component and accumulate
    for var in comps:
        irr_var, noi_var = loader(var)
        irr_parts.append(irr_var)
        noi_parts.append(noi_var)

    # sum all parts to get total ET
    total_irr = sum(irr_parts)
    total_noi = sum(noi_parts)

    evapotran[model] = {
        'IRR': total_irr,
        'NOI': total_noi
    }
    
# specify the TWS variable for each model
TWS_VARS = {
    'CESM2':    'TWS',
    'CESM2_gw': 'TWS',
    'NorESM':   'TWS',
    'E3SM':     'TWS',
    'IPSL':     'mrtws',
    'CNRM':     'mrtws',
    'MIROC':    'tws',
}

# container for all TWS datasets
tws = {}

for model, var in TWS_VARS.items():
    loader = MODEL_LOADERS[model]
    irr_tws, noi_tws = loader(var)
    tws[model] = {
        'IRR': irr_tws,
        'NOI': noi_tws
    }

In [3]:
DATA_GROUPS = {
    'P'  : precip,
    'R'  : runoff,
    'TWS': tws,
    'ET' : evapotran,
}

# Here we use CESM2 output for masking in case some models also output grid cells over the ocean

for name, group in DATA_GROUPS.items():
    # make mask from CESM2 IRR
    ref_mask = np.isnan(group['CESM2']['IRR'])
    # apply to every other model & both scenarios
    for model, scen_dict in group.items():
        if model == 'CESM2':
            continue
        for scen in ('IRR','NOI'):
            scen_dict[scen][ref_mask] = np.nan

In [7]:
def calcu_global_water(data): # This is the function to calculate global mean water fluxes (not used here in this script)
    
    str_area = '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/input_data/AREA.mat'
    
    area = get_data_from_mat(str_area, 'AREA')
    
    area_for_calcu = area.T
    
    area_for_calcu[np.isnan(data[0, :, :])] = np.nan
    
    
    data_globe = data * AREA.T
    
    data_globe = np.nansum(data_globe, axis=(1, 2))
    
    area_for_calcu = np.nansum(area_for_calcu, axis=(0, 1))
    
    return data_globe/area_for_calcu

def calcu_water_region(data, region_id): 
    
    # This is the function to calculate regional mean water fluxes (used here)
    
    str_area = '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/input_data/AREA.mat'
    
    area = get_data_from_mat(str_area, 'AREA') 
    
    # Area for CLM_data, we read it every time when calling this function
    # Because in Jupyter notebook the array will be changed even though only being used in another function
    
    area_for_calcu = area.T
    
    # build a mask of all points to set to NaN
    mask = (
        np.isnan(data[0, :, :])                                    # missing data
        | (np.abs(ar6_region - region_id) > 0.2)                   # outside [region_id–tol, region_id+tol]
    )
    
    area_for_calcu[mask] = np.nan

    # Keep only the area in the region
    
    data_region = data * area_for_calcu
    
    data_region[:, mask] = np.nan

    data_region = np.nansum(data_region, axis=(1, 2))

    area_for_calcu = np.nansum(area_for_calcu, axis=(0, 1))
    
    return data_region/area_for_calcu

In [20]:
def get_regional_p_et(region_id):
    
    def _calc_p_et_anomalies(model_key, region_id, baseline_end=114):
        """
        Returns (irr_anomaly, noi_anomaly) time‐series for the given model.
        """
        # compute raw P–ET and aggregate over the region
        
        irr = calcu_water_region(
            precip[model_key]['IRR'] - evapotran[model_key]['IRR'],
            region_id
        )
        
        noi = calcu_water_region(
            precip[model_key]['NOI'] - evapotran[model_key]['NOI'],
            region_id
        )
        
        # baseline from the first `baseline_end` timesteps of the non‑irrigated run
        baseline = np.mean(noi[:baseline_end])
        
        return irr - baseline, noi - baseline
    
    # 1. Define the models in the exact order you want them stacked
    models = [
        'CESM2',
        'CESM2_gw',
        'NorESM',
        'E3SM',
        'IPSL',
        'MIROC',
        'CNRM',
    ]

    # 2. Prepare empty lists to collect each model's IRR/NOI series
    irr_list = []
    noi_list = []

    # 3. Loop once, calculate both series (using your helper), and append
    for m in models:
        irr_ts, noi_ts = _calc_p_et_anomalies(m, region_id)
        irr_list.append(irr_ts)
        noi_list.append(noi_ts)

    # 4. Stack them all in one go – same as your vstack calls
    data_irr_p_et = np.vstack(irr_list)
    data_noi_p_et = np.vstack(noi_list)

    # 5. Compute the difference
    data_diff_p_et = data_irr_p_et - data_noi_p_et
    
    return data_irr_p_et, data_noi_p_et, data_diff_p_et

In [21]:
# 1) map your region names → AR6 IDs
REGIONS = {
    'SAS': 38,
    'MED': 20,
    'CNA':  5,
    'WCA': 33,
}

def compute_stats(arr: np.ndarray, axis=0) -> dict[str, np.ndarray]:
    """
    Return the 25th, 50th, 75th percentiles and mean of `arr` along `axis`.
    """
    pcts = np.percentile(arr, [25, 50, 75], axis=axis)
    return {
        'p25': pcts[0],
        'p50': pcts[1],
        'p75': pcts[2],
        'mean': np.mean(arr, axis=axis),
    }

def batch_regional_stats(regions: dict[str,int]):
    """
    For each region in `regions`, call get_regional_P_E and compute stats
    for IRR, NOI, and their difference.
    Returns a nested dict: stats[region]['irr']['p75'], etc.
    """
    stats: dict[str, dict[str, dict[str, np.ndarray]]] = {}

    for name, rid in regions.items():
        irr_arr, noi_arr, diff_arr = get_regional_p_et(rid)

        stats[name] = {
            'irr':  compute_stats(irr_arr),
            'noi':  compute_stats(noi_arr),
            'diff': compute_stats(diff_arr),
        }

    return stats

# run it once
region_stats = batch_regional_stats(REGIONS)

# then extract whatever you need:
# data_irr_p_minus_et_SAS_75  = region_stats['SAS']['irr']['p75']
# data_irr_p_minus_et_SAS_50  = region_stats['SAS']['irr']['p50']
# data_irr_p_minus_et_SAS_25  = region_stats['SAS']['irr']['p25']
# data_irr_p_minus_et_SAS_mean = region_stats['SAS']['irr']['mean']

In [27]:
str_lat_lon = '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/surfdata_irrigation_method.nc'

data_surface = Data_from_nc(str_lat_lon)   #load the data containing lat and lon information for plottings


data_lon = data_surface.load_variable('LONGXY')

data_lon = data_lon[0,:]

data_lat = data_surface.load_variable('LATIXY')

data_lat = data_lat[29:,0]

data_lat = data_lat[:]

def plot_line(ax, index, data_mean, data_min, data_max, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=16, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 4, marker = marker, markersize=1.2, alpha=0.8)
    ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right', fontsize = 18)
    plt.title(title1, loc='left', fontsize = 18)
    plt.ylabel(ylabel, fontsize = 16)
    plt.xlabel(xlabel, fontsize = 16)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize=14)
    plt.xlim(1901,2014)

In [253]:
# calcu the confidence interval for the slope of linear relashionship
def conf_interval(theta_hat, se, n, alpha=0.05):
    
    df = n - 1
    
    t_star = stats.t.ppf(1 - alpha/2, df)
    
    lower = theta_hat - t_star * se
    
    upper = theta_hat + t_star * se
    
    return lower, upper

# we always smoothen the data first
def linear_regression_and_confidence_interval(data, X=np.array(range(1901,2015))): 
    
    params = linregress(X,savgol_filter((data),19,2))
    
    slope = params[0]
    
    intercept = params[1]
    
    rvalue = params[2]
    
    pvalue = params[3]
    
    stderr = params[4]
    
    b_hat, se_b, n = slope, stderr, len(data)-1
    
    lo, hi = conf_interval(b_hat, se_b, n)
    
    return [slope, intercept, rvalue, pvalue, hi-slope]

# This is where we calculate the linear regression parameters

param_irr_SAS1_p_et = linear_regression_and_confidence_interval(region_stats['SAS']['irr']['mean'][:60], X=np.array(range(1901,1961)))

param_noi_SAS1_p_et = linear_regression_and_confidence_interval(region_stats['SAS']['noi']['mean'][:60], X=np.array(range(1901,1961)))

param_irr_SAS2_p_et = linear_regression_and_confidence_interval(region_stats['SAS']['irr']['mean'][60:], X=np.array(range(1961,2015)))

param_noi_SAS2_p_et = linear_regression_and_confidence_interval(region_stats['SAS']['noi']['mean'][60:], X=np.array(range(1961,2015)))

param_diff_SAS1_p_et = linear_regression_and_confidence_interval(region_stats['SAS']['diff']['mean'][:60], X=np.array(range(1901,1961)))

param_diff_SAS2_p_et = linear_regression_and_confidence_interval(region_stats['SAS']['diff']['mean'][60:], X=np.array(range(1961,2015)))



param_irr_MED1_p_et = linear_regression_and_confidence_interval(region_stats['MED']['irr']['mean'][:60], X=np.array(range(1901,1961)))

param_noi_MED1_p_et = linear_regression_and_confidence_interval(region_stats['MED']['noi']['mean'][:60], X=np.array(range(1901,1961)))

param_diff_MED1_p_et = linear_regression_and_confidence_interval(region_stats['MED']['diff']['mean'][:60], X=np.array(range(1901,1961)))

param_irr_MED2_p_et = linear_regression_and_confidence_interval(region_stats['MED']['irr']['mean'][60:], X=np.array(range(1961,2015)))

param_noi_MED2_p_et = linear_regression_and_confidence_interval(region_stats['MED']['noi']['mean'][60:], X=np.array(range(1961,2015)))

param_diff_MED2_p_et = linear_regression_and_confidence_interval(region_stats['MED']['diff']['mean'][60:], X=np.array(range(1961,2015)))



param_irr_CNA1_p_et = linear_regression_and_confidence_interval(region_stats['CNA']['irr']['mean'])

param_noi_CNA1_p_et = linear_regression_and_confidence_interval(region_stats['CNA']['noi']['mean'])

param_diff_CNA1_p_et = linear_regression_and_confidence_interval(region_stats['CNA']['diff']['mean'])



param_irr_WCA1_p_et = linear_regression_and_confidence_interval(region_stats['WCA']['irr']['mean'])

param_noi_WCA1_p_et = linear_regression_and_confidence_interval(region_stats['WCA']['noi']['mean'])

param_diff_WCA1_p_et = linear_regression_and_confidence_interval(region_stats['WCA']['diff']['mean'])


# We can print param_diff_WCA to get all parameters

In [ ]:
fig = plt.figure(figsize=(12, 20),dpi=300)

fig.subplots_adjust(hspace=0.2, wspace=0.3, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)

##############################
# Subfigure 1, SAS irr and noi

ax1 = plt.subplot(421, frameon=True)
index = 'a'
plot_line(ax1, index, savgol_filter((region_stats['SAS']['irr']['mean']),19,2), savgol_filter((region_stats['SAS']['irr']['p25']),19,2), savgol_filter((region_stats['SAS']['irr']['p75']),19,2), r'South Asia', '', 'tranirr', 'dodgerblue', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((region_stats['SAS']['noi']['mean']),19,2), savgol_filter((region_stats['SAS']['noi']['p25']),19,2), savgol_filter((region_stats['SAS']['noi']['p75']),19,2), r'South Asia', '', '1901irr', 'brown', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
x1 = np.array(range(1901, 1961))

param_irr_1_plot = param_irr_SAS1_p_et
param_noi_1_plot = param_noi_SAS1_p_et

y1_irr = param_irr_1_plot[0] * x1 + param_irr_1_plot[1]
y1_noi = param_noi_1_plot[0] * x1 + param_noi_1_plot[1]

x2 = np.array(range(1961, 2015))

param_irr_2_plot = param_irr_SAS2_p_et
param_noi_2_plot = param_noi_SAS2_p_et

y2_irr = param_irr_2_plot[0] * x2 + param_irr_2_plot[1]
y2_noi = param_noi_2_plot[0] * x2 + param_noi_2_plot[1]

plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[10], y1_irr[30]-90, r'+0.776***($\pm$0.215)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[10], y1_noi[10]+80, '+1.194***($\pm$0.148)', fontsize = 14, color = 'brown', weight='bold')
plt.legend(loc = 'upper left', fontsize = 14)
plt.plot(x2, y2_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x2[0], y2_irr[10]-100, r'-1.461***($\pm$0.261)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x2, y2_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x2[0], y2_noi[10]+60, '-0.664***($\pm$0.283)', fontsize = 14, color = 'brown', weight='bold')
plt.grid(linestyle = '--')
plt.ylim([-300, 300])
x1 = np.array(range(1901, 2015))
y1_irr_noi = -0.5750347087001787 * x1 + 1095.8782203820022

##############################
# Subfigure 2, SAS dif

ax1 = plt.subplot(422, frameon=True)
index = 'b'
plot_line(ax1, index, savgol_filter((region_stats['SAS']['diff']['mean']),19,2), savgol_filter((region_stats['SAS']['diff']['p25']),19,2), savgol_filter((region_stats['SAS']['diff']['p75']),19,2), r'South Asia', '', 'tranirr-1901irr', 'green', r'land $\Delta$(P-ET) ($\mathregular{mm/year}$)', '', '')
plt.grid(linestyle = '--')
x1 = np.array(range(1901, 1961))

param_diff_1_plot = param_diff_SAS1_p_et


y1_diff = param_diff_1_plot[0] * x1 + param_diff_1_plot[1]


x2 = np.array(range(1961, 2015))

param_diff_2_plot = param_diff_SAS2_p_et


y2_diff = param_diff_2_plot[0] * x2 + param_diff_2_plot[1]


plt.plot(x1, y1_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.legend(loc = 'best', fontsize = 14)
plt.plot(x2, y2_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.text(x1[10], y1_irr[30]-50, r'-0.418***($\pm$0.208)', fontsize = 14, color = 'green', weight='bold')
plt.text(x2[0], y2_irr[50]-50, r'-0.797***($\pm$0.159)', fontsize = 14, color = 'green', weight='bold')
plt.ylim([-200, 200])



##############################
# Subfigure 3, MED irr and noi

ax1 = plt.subplot(423, frameon=True)
index = 'c'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((region_stats['MED']['irr']['mean']),19,2), savgol_filter((region_stats['MED']['irr']['p25']),19,2), savgol_filter((region_stats['MED']['irr']['p75']),19,2), r'Mediterranean', '', 'tranirr', 'dodgerblue', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((region_stats['MED']['noi']['mean']),19,2), savgol_filter((region_stats['MED']['noi']['p25']),19,2), savgol_filter((region_stats['MED']['noi']['p75']),19,2), r'Mediterranean', '', '1901irr', 'brown', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')

x1 = np.array(range(1901, 1961))

param_irr_1_plot = param_irr_MED1_p_et
param_noi_1_plot = param_noi_MED1_p_et

y1_irr = param_irr_1_plot[0] * x1 + param_irr_1_plot[1]
y1_noi = param_noi_1_plot[0] * x1 + param_noi_1_plot[1]

x2 = np.array(range(1961, 2015))

param_irr_2_plot = param_irr_MED2_p_et
param_noi_2_plot = param_noi_MED2_p_et

y2_irr = param_irr_2_plot[0] * x2 + param_irr_2_plot[1]
y2_noi = param_noi_2_plot[0] * x2 + param_noi_2_plot[1]

plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[10], y1_irr[30]+15, r'-0.008($\pm$0.055)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[10], y1_noi[10]-20, '+0.070***($\pm$0.029)', fontsize = 14, color = 'brown', weight='bold')
plt.legend(loc = 'upper left', fontsize = 14)
plt.plot(x2, y2_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x2[0], y2_irr[10]-30, r'-0.425***($\pm$0.068)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x2, y2_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x2[0], y2_noi[10]+20, '-0.088*($\pm$0.093)', fontsize = 14, color = 'brown', weight='bold')


plt.grid(linestyle = '--')
plt.ylim([-80, 80])

##############################
# Subfigure 4, MED diff

ax1 = plt.subplot(424, frameon=True)
index = 'd'
plot_line(ax1, index, savgol_filter((region_stats['MED']['diff']['mean']),19,2), savgol_filter((region_stats['MED']['diff']['p25']),19,2), savgol_filter((region_stats['MED']['diff']['p75']),19,2), r'Mediterranean', '', 'tranirr-1901irr', 'green', r'land $\Delta$(P-ET) ($\mathregular{mm/year}$)', '', '')
plt.grid(linestyle = '--')
x1 = np.array(range(1901, 1961))

param_diff_1_plot = param_diff_MED1_p_et


y1_diff = param_diff_1_plot[0] * x1 + param_diff_1_plot[1]


x2 = np.array(range(1961, 2015))

param_diff_2_plot = param_diff_MED2_p_et


y2_diff = param_diff_2_plot[0] * x2 + param_diff_2_plot[1]


plt.plot(x1, y1_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.legend(loc = 'best', fontsize = 14)
plt.plot(x2, y2_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)

plt.text(x1[10], y1_irr_noi[10]+20, r'-0.078***($\pm$0.057)', fontsize = 14, color = 'green', weight='bold')
plt.text(x2[0], y2_irr[50]-20, r'-0.338***($\pm$0.064)', fontsize = 14, color = 'green', weight='bold')

plt.ylim([-80, 80])


##############################
# Subfigure 5, CNA irr and noi

ax1 = plt.subplot(425, frameon=True)
index = 'e'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((region_stats['CNA']['irr']['mean']),19,2), savgol_filter((region_stats['CNA']['irr']['p25']),19,2), savgol_filter((region_stats['CNA']['irr']['p75']),19,2), r'Central North America', '', 'tranirr', 'dodgerblue', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((region_stats['CNA']['noi']['mean']),19,2), savgol_filter((region_stats['CNA']['noi']['p25']),19,2), savgol_filter((region_stats['CNA']['noi']['p75']),19,2), r'Central North America', '', '1901irr', 'brown', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')

x1 = np.array(range(1901, 2015))

param_irr_1_plot = param_irr_CNA1_p_et
param_noi_1_plot = param_noi_CNA1_p_et

y1_irr = param_irr_1_plot[0] * x1 + param_irr_1_plot[1]
y1_noi = param_noi_1_plot[0] * x1 + param_noi_1_plot[1]



plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[60], y1_irr[30]-50, r'-0.299***($\pm$0.031)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[10], y1_noi[10]+20, '-0.209***($\pm$0.048)', fontsize = 14, color = 'brown', weight='bold')

plt.legend(loc = 'best', fontsize = 14)
plt.grid(linestyle = '--')
plt.ylim([-100, 100])

##############################
# Subfigure 6, CNA diff

ax1 = plt.subplot(426, frameon=True)
index = 'f'

x1 = np.array(range(1901, 2015))
param_diff_1_plot = param_diff_CNA1_p_et


y1_diff = param_diff_1_plot[0] * x1 + param_diff_1_plot[1]
plot_line(ax1, index, savgol_filter((region_stats['CNA']['diff']['mean']),19,2), savgol_filter((region_stats['CNA']['diff']['p25']),19,2), savgol_filter((region_stats['CNA']['diff']['p75']),19,2), r'Central North America', '', 'tranirr-1901irr', 'green', r'land $\Delta$(P-ET) ($\mathregular{mm/year}$)', '', '')

plt.plot(x1, y1_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.text(x1[40], y1_irr_noi[60]-10, r'-0.091***($\pm$0.041)', fontsize = 14, color = 'green', weight='bold')
plt.grid(linestyle = '--')
plt.legend(loc = 'best', fontsize = 14)
plt.ylim([-100, 100])

##############################
# Subfigure 7, WCA irr and noi

ax1 = plt.subplot(427, frameon=True)
index = 'g'
x1 = np.array(range(1901, 2015))

param_irr_1_plot = param_irr_WCA1_p_et
param_noi_1_plot = param_noi_WCA1_p_et

y1_irr = param_irr_1_plot[0] * x1 + param_irr_1_plot[1]
y1_noi = param_noi_1_plot[0] * x1 + param_noi_1_plot[1]


plot_line(ax1, index, savgol_filter((region_stats['WCA']['irr']['mean']),19,2), savgol_filter((region_stats['WCA']['irr']['p25']),19,2), savgol_filter((region_stats['WCA']['irr']['p75']),19,2), r'West Central Asia', '', 'tranirr', 'dodgerblue', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((region_stats['WCA']['noi']['mean']),19,2), savgol_filter((region_stats['WCA']['noi']['p25']),19,2), savgol_filter((region_stats['WCA']['noi']['p75']),19,2), r'West Central Asia', '', '1901irr', 'brown', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')

plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[10], y1_irr[30]-20, r'-0.116***($\pm$0.021)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[10], y1_noi[10]+20, '+0.023***($\pm$0.017)', fontsize = 14, color = 'brown', weight='bold')

# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plt.legend(loc = 'best', fontsize = 14)

plt.grid(linestyle = '--')
plt.ylim([-50, 50])

##############################
# Subfigure 8, WCA diff

ax1 = plt.subplot(428, frameon=True)
index = 'h'
x1 = np.array(range(1901, 2015))
param_diff_1_plot = param_diff_WCA1_p_et


y1_diff = param_diff_1_plot[0] * x1 + param_diff_1_plot[1]
plot_line(ax1, index, savgol_filter((region_stats['WCA']['diff']['mean']),19,2), savgol_filter((region_stats['WCA']['diff']['p25']),19,2), savgol_filter((region_stats['WCA']['diff']['p75']),19,2), r'West Central Asia', '', 'tranirr-1901irr', 'green', r'land $\Delta$(P-ET) ($\mathregular{mm/year}$)', '', '')
plt.plot(x1, y1_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.text(x1[40], y1_irr_noi[60]-5, r'-0.140***($\pm$0.019)', fontsize = 14, color = 'green', weight='bold')


plt.grid(linestyle = '--')
plt.legend(loc = 'best', fontsize = 14)
plt.ylim([-50, 50])




In [118]:
# Here we get regional tws data.

def get_regional_tws(region_id):

    data_irr_cesm2_gw_tws = calcu_water_region(tws['CESM2_gw']['IRR'], region_id)
    data_noi_cesm2_gw_tws = calcu_water_region(tws['CESM2_gw']['NOI'], region_id)
    data_irr_cesm2_gw_tws = data_irr_cesm2_gw_tws - data_noi_cesm2_gw_tws[0]
    data_noi_cesm2_gw_tws = data_noi_cesm2_gw_tws - data_noi_cesm2_gw_tws[0]

    data_irr_cnrm_tws = calcu_water_region(tws['CNRM']['IRR'], region_id)
    data_noi_cnrm_tws = calcu_water_region(tws['CNRM']['NOI'], region_id)
    data_irr_cnrm_tws = data_irr_cnrm_tws - data_noi_cnrm_tws[0]
    data_noi_cnrm_tws = data_noi_cnrm_tws - data_noi_cnrm_tws[0]
    
    data_irr_miroc_tws = calcu_water_region(tws['MIROC']['IRR'], region_id)
    data_noi_miroc_tws = calcu_water_region(tws['MIROC']['NOI'], region_id)
    data_irr_miroc_tws = data_irr_miroc_tws - data_noi_miroc_tws[0]
    data_noi_miroc_tws = data_noi_miroc_tws - data_noi_miroc_tws[0]
    
    data_irr_ipsl_tws = calcu_water_region(tws['IPSL']['IRR'], region_id)
    data_noi_ipsl_tws = calcu_water_region(tws['IPSL']['NOI'], region_id)
    data_irr_ipsl_tws = data_irr_ipsl_tws - data_noi_ipsl_tws[0]
    data_noi_ipsl_tws = data_noi_ipsl_tws - data_noi_ipsl_tws[0]
    
    data_irr_e3sm_tws = calcu_water_region(tws['E3SM']['IRR'], region_id)
    data_noi_e3sm_tws = calcu_water_region(tws['E3SM']['NOI'], region_id)
    data_irr_e3sm_tws = data_irr_e3sm_tws - data_noi_e3sm_tws[0]
    data_noi_e3sm_tws = data_noi_e3sm_tws - data_noi_e3sm_tws[0]
    
    data_irr_cesm2_tws = calcu_water_region(tws['CESM2']['IRR'], region_id)
    data_noi_cesm2_tws = calcu_water_region(tws['CESM2']['NOI'], region_id)
    data_irr_cesm2_tws = data_irr_cesm2_tws - data_noi_cesm2_tws[0]
    data_noi_cesm2_tws = data_noi_cesm2_tws - data_noi_cesm2_tws[0]

    data_irr_noresm_tws = calcu_water_region(tws['NorESM']['IRR'], region_id)
    data_noi_noresm_tws = calcu_water_region(tws['NorESM']['NOI'], region_id)
    data_irr_noresm_tws = data_irr_noresm_tws - data_noi_noresm_tws[0]
    data_noi_noresm_tws = data_noi_noresm_tws - data_noi_noresm_tws[0]
    
    

    data_irr_all_tws_grd = np.vstack((
                               data_irr_cesm2_gw_tws,
                               data_irr_miroc_tws,
                               data_irr_cnrm_tws,
                                data_irr_ipsl_tws,
                                data_irr_e3sm_tws))
    
    data_irr_all_tws_grd_75 = np.percentile(data_irr_all_tws_grd, 100, axis = 0)
    data_irr_all_tws_grd_25 = np.percentile(data_irr_all_tws_grd, 0, axis = 0)
    data_irr_all_tws_grd_50 = np.percentile(data_irr_all_tws_grd, 50, axis = 0)
    data_irr_all_tws_grd_mean = np.mean(data_irr_all_tws_grd, axis = 0)
    


    data_noi_all_tws_grd = np.vstack((
                                   data_noi_cesm2_gw_tws,
                                   data_noi_miroc_tws,
                                   data_noi_cnrm_tws,
                                    data_noi_ipsl_tws,
                                    data_noi_e3sm_tws))
    
    data_noi_all_tws_grd_75 = np.percentile(data_noi_all_tws_grd, 100, axis = 0)
    data_noi_all_tws_grd_25 = np.percentile(data_noi_all_tws_grd, 0, axis = 0)
    data_noi_all_tws_grd_50 = np.percentile(data_noi_all_tws_grd, 50, axis = 0)
    data_noi_all_tws_grd_mean = np.mean(data_noi_all_tws_grd, axis = 0)
    
    data_diff_all_tws_grd = data_irr_all_tws_grd - data_noi_all_tws_grd
    
    data_diff_all_tws_grd_75 = np.percentile(data_diff_all_tws_grd, 100, axis = 0)
    data_diff_all_tws_grd_25 = np.percentile(data_diff_all_tws_grd, 0, axis = 0)
    data_diff_all_tws_grd_50 = np.percentile(data_diff_all_tws_grd, 50, axis = 0)
    data_diff_all_tws_grd_mean = np.mean(data_diff_all_tws_grd, axis = 0)
    
    data_irr_all_tws_nogrd = np.vstack((
                                data_irr_cesm2_tws,
                                data_irr_noresm_tws))
    data_irr_all_tws_nogrd_mean = np.mean(data_irr_all_tws_nogrd, axis = 0)
    
    data_noi_all_tws_nogrd = np.vstack((
                                data_noi_cesm2_tws,
                                data_noi_noresm_tws))
    data_noi_all_tws_nogrd_mean = np.mean(data_noi_all_tws_nogrd, axis = 0)
    
    data_diff_all_tws_nogrd = data_irr_all_tws_nogrd - data_noi_all_tws_nogrd
    
    data_diff_all_tws_nogrd_mean = np.mean(data_diff_all_tws_nogrd, axis = 0)
    
    data_irr_all_tws = np.vstack((
                               data_irr_all_tws_grd_75,
                               data_irr_all_tws_grd_25,
                               data_irr_all_tws_grd_mean,
                                data_irr_ipsl_tws,
                                data_irr_e3sm_tws,
                                data_irr_all_tws_nogrd))
    
    data_noi_all_tws = np.vstack((
                               data_noi_all_tws_grd_75,
                               data_noi_all_tws_grd_25,
                               data_noi_all_tws_grd_mean,
                                data_noi_ipsl_tws,
                                data_noi_e3sm_tws,
                                data_noi_all_tws_nogrd))
    
    data_diff_all_tws = np.vstack((
                               data_diff_all_tws_grd_75,
                               data_diff_all_tws_grd_25,
                               data_diff_all_tws_grd_mean,
                                data_irr_ipsl_tws - data_noi_ipsl_tws,
                                data_irr_e3sm_tws - data_noi_e3sm_tws,
                                data_diff_all_tws_nogrd))
    
    return data_irr_all_tws, data_noi_all_tws, data_diff_all_tws

In [119]:
data_irr_all_tws_SAS, data_noi_all_tws_SAS, data_diff_all_tws_SAS = get_regional_tws(38)
data_irr_all_tws_MED, data_noi_all_tws_MED, data_diff_all_tws_MED = get_regional_tws(20)
data_irr_all_tws_CNA, data_noi_all_tws_CNA, data_diff_all_tws_CNA = get_regional_tws(5)
data_irr_all_tws_WCA, data_noi_all_tws_WCA, data_diff_all_tws_WCA = get_regional_tws(33)

In [323]:
param_irr_SAS1 = linear_regression_and_confidence_interval(savgol_filter(data_irr_all_tws_SAS[2,:60]/365/86400,19,2), X=np.array(range(1901,1961)))
param_noi_SAS1 = linear_regression_and_confidence_interval(savgol_filter(data_noi_all_tws_SAS[2,:60]/365/86400,19,2), X=np.array(range(1901,1961)))

param_irr_SAS2 = linear_regression_and_confidence_interval(savgol_filter(data_irr_all_tws_SAS[2,60:]/365/86400,19,2), X=np.array(range(1961,2015)))
param_noi_SAS2 = linear_regression_and_confidence_interval(savgol_filter(data_noi_all_tws_SAS[2,60:]/365/86400,19,2), X=np.array(range(1961,2015)))

param_diff_SAS1 = linear_regression_and_confidence_interval(savgol_filter(data_diff_all_tws_SAS[2,:60]/365/86400,19,2), X=np.array(range(1901,1961)))
param_diff_SAS2 = linear_regression_and_confidence_interval(savgol_filter(data_diff_all_tws_SAS[2,60:]/365/86400,19,2), X=np.array(range(1961,2015)))


In [337]:
param_irr_MED1 = linear_regression_and_confidence_interval(savgol_filter(data_irr_all_tws_MED[2,:60]/365/86400,19,2), X=np.array(range(1901,1961)))
param_noi_MED1 = linear_regression_and_confidence_interval(savgol_filter(data_noi_all_tws_MED[2,:60]/365/86400,19,2), X=np.array(range(1901,1961)))

param_irr_MED2 = linear_regression_and_confidence_interval(savgol_filter(data_irr_all_tws_MED[2,60:]/365/86400,19,2), X=np.array(range(1961,2015)))
param_noi_MED2 = linear_regression_and_confidence_interval(savgol_filter(data_noi_all_tws_MED[2,60:]/365/86400,19,2), X=np.array(range(1961,2015)))

param_diff_MED1 = linear_regression_and_confidence_interval(savgol_filter(data_diff_all_tws_MED[2,:60]/365/86400,19,2), X=np.array(range(1901,1961)))
param_diff_MED2 = linear_regression_and_confidence_interval(savgol_filter(data_diff_all_tws_MED[2,60:]/365/86400,19,2), X=np.array(range(1961,2015)))


In [334]:
param_irr_CNA1 = linear_regression_and_confidence_interval(savgol_filter(data_irr_all_tws_CNA[2,:20]/365/86400,19,2), X=np.array(range(1901,1921)))
param_noi_CNA1 = linear_regression_and_confidence_interval(savgol_filter(data_noi_all_tws_CNA[2,:20]/365/86400,19,2), X=np.array(range(1901,1921)))

param_irr_CNA2 = linear_regression_and_confidence_interval(savgol_filter(data_irr_all_tws_CNA[2,20:]/365/86400,19,2), X=np.array(range(1921,2015)))
param_noi_CNA2 = linear_regression_and_confidence_interval(savgol_filter(data_noi_all_tws_CNA[2,20:]/365/86400,19,2), X=np.array(range(1921,2015)))

param_diff_CNA1 = linear_regression_and_confidence_interval(savgol_filter(data_diff_all_tws_CNA[2,:20]/365/86400,19,2), X=np.array(range(1901,1921)))
param_diff_CNA2 = linear_regression_and_confidence_interval(savgol_filter(data_diff_all_tws_CNA[2,20:]/365/86400,19,2), X=np.array(range(1921,2015)))


In [349]:
param_irr_WCA1 = linear_regression_and_confidence_interval(savgol_filter(data_irr_all_tws_WCA[2,:60]/365/86400,19,2), X=np.array(range(1901,1961)))
param_noi_WCA1 = linear_regression_and_confidence_interval(savgol_filter(data_noi_all_tws_WCA[2,:60]/365/86400,19,2), X=np.array(range(1901,1961)))

param_irr_WCA2 = linear_regression_and_confidence_interval(savgol_filter(data_irr_all_tws_WCA[2,60:]/365/86400,19,2), X=np.array(range(1961,2015)))
param_noi_WCA2 = linear_regression_and_confidence_interval(savgol_filter(data_noi_all_tws_WCA[2,60:]/365/86400,19,2), X=np.array(range(1961,2015)))

param_diff_WCA1 = linear_regression_and_confidence_interval(savgol_filter(data_diff_all_tws_WCA[2,:60]/365/86400,19,2), X=np.array(range(1901,1961)))
param_diff_WCA2 = linear_regression_and_confidence_interval(savgol_filter(data_diff_all_tws_WCA[2,60:]/365/86400,19,2), X=np.array(range(1961,2015)))

In [ ]:
def plot_line_no_grd(ax, index, data_mean, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=16, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 2, marker = marker, markersize=1.2, alpha=0.4, linestyle='--')
#     ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right', fontsize = 18)
    plt.title(title1, loc='left', fontsize = 18)
    plt.ylabel(ylabel, fontsize = 16)
    plt.xlabel(xlabel, fontsize = 16)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize=14)
    plt.xlim(1901,2014)
    plt.legend(loc = 'best', fontsize = 14)
    
def plot_line_IPSL(ax, index, data_mean, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=16, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 2, marker = marker, markersize=1.2, alpha=0.4, linestyle=':')
#     ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right', fontsize = 18)
    plt.title(title1, loc='left', fontsize = 18)
    plt.ylabel(ylabel, fontsize = 16)
    plt.xlabel(xlabel, fontsize = 16)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize=14)
    plt.xlim(1901,2014)
    plt.legend(loc = 'best', fontsize = 14)
    
def plot_line_E3SM(ax, index, data_mean, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=16, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 2, marker = marker, markersize=1.2, alpha=0.4, linestyle='-.')
#     ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right', fontsize = 18)
    plt.title(title1, loc='left', fontsize = 18)
    plt.ylabel(ylabel, fontsize = 16)
    plt.xlabel(xlabel, fontsize = 16)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize=14)
    plt.xlim(1901,2014)
    plt.legend(loc = 'best', fontsize = 14)


fig = plt.figure(figsize=(12, 20),dpi=300)
fig.subplots_adjust(hspace=0.2, wspace=0.3, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)





ax1 = plt.subplot(421, frameon=True)
index = 'a'
# plot_line(ax1, index, savgol_filter((data_noi_all_tws_mean),19,2), savgol_filter((data_noi_all_tws[1,:]),19,2), savgol_filter((data_noi_all_tws[0,:]),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((data_irr_all_tws_SAS[2,:])/365/86400,19,2), savgol_filter((data_irr_all_tws_SAS[1,:])/365/86400,19,2), savgol_filter((data_irr_all_tws_SAS[0,:])/365/86400,19,2), r'South Asia', '', 'tranirr', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line(ax1, index, savgol_filter((data_noi_all_tws_SAS[2,:])/365/86400,19,2), savgol_filter((data_noi_all_tws_SAS[1,:])/365/86400,19,2), savgol_filter((data_noi_all_tws_SAS[0,:])/365/86400,19,2), r'South Asia', '', '1901irr', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_no_grd(ax1, index, savgol_filter((data_irr_all_tws_SAS[5,:])/365/86400,19,2), r'South Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((data_noi_all_tws_SAS[5,:])/365/86400,19,2), r'South Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

# plot_line_IPSL(ax1, index, savgol_filter((data_irr_all_tws_SAS[3,:])/365/86400,19,2), r'South Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
# plot_line_IPSL(ax1, index, savgol_filter((data_noi_all_tws_SAS[3,:])/365/86400,19,2), r'South Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

# plot_line_E3SM(ax1, index, savgol_filter((data_irr_all_tws_SAS[4,:])/365/86400,19,2), r'South Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
# plot_line_E3SM(ax1, index, savgol_filter((data_noi_all_tws_SAS[4,:])/365/86400,19,2), r'South Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')
x1 = np.array(range(1901, 1961))

param_irr_1_plot = param_irr_SAS1
param_noi_1_plot = param_noi_SAS1

y1_irr = param_irr_1_plot[0] * x1 + param_irr_1_plot[1]
y1_noi = param_noi_1_plot[0] * x1 + param_noi_1_plot[1]

x2 = np.array(range(1961, 2015))

param_irr_2_plot = param_irr_SAS2
param_noi_2_plot = param_noi_SAS2

y2_irr = param_irr_2_plot[0] * x2 + param_irr_2_plot[1]
y2_noi = param_noi_2_plot[0] * x2 + param_noi_2_plot[1]

plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[5], y1_irr[30]-900, r'-6.453***($\pm$0.127)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[5], y1_noi[10]+800, '-2.788***($\pm$0.074)', fontsize = 14, color = 'brown', weight='bold')
plt.legend(loc = 'upper left', fontsize = 14)
plt.plot(x2, y2_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x2[0], y2_irr[10]-1200, r'-16.008***($\pm$0.557)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x2, y2_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x2[0], y2_noi[10]+600, '-2.559***($\pm$0.094)', fontsize = 14, color = 'brown', weight='bold')

plt.grid(linestyle = '--')
plt.ylim([-5000, 5000])

ax1 = plt.subplot(422, frameon=True)
index = 'b'

x1 = np.array(range(1901, 1961))

param_diff_1_plot = param_diff_SAS1


y1_diff = param_diff_1_plot[0] * x1 + param_diff_1_plot[1]


x2 = np.array(range(1961, 2015))

param_diff_2_plot = param_diff_SAS2


y2_diff = param_diff_2_plot[0] * x2 + param_diff_2_plot[1]

plot_line(ax1, index, savgol_filter((data_diff_all_tws_SAS[2,:])/365/86400,19,2), savgol_filter((data_diff_all_tws_SAS[1,:])/365/86400,19,2), savgol_filter((data_diff_all_tws_SAS[0,:])/365/86400,19,2), r'South Asia', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((data_diff_all_tws_SAS[5,:])/365/86400,19,2), r'South Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plot_line_IPSL(ax1, index, savgol_filter((data_diff_all_tws_SAS[3,:])/365/86400,19,2), r'South Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plot_line_E3SM(ax1, index, savgol_filter((data_diff_all_tws_SAS[4,:])/365/86400,19,2), r'South Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plt.plot(x1, y1_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.legend(loc = 'best', fontsize = 14)
plt.plot(x2, y2_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.text(x1[10], y1_diff[30]-1000, r'-3.665***($\pm$0.142)', fontsize = 14, color = 'green', weight='bold')
plt.text(x2[0], y2_diff[50]-1500, r'-13.449***($\pm$0.594)', fontsize = 14, color = 'green', weight='bold')
plt.grid(linestyle = '--')
plt.ylim([-4000, 4000])




ax1 = plt.subplot(423, frameon=True)
index = 'c'
# plot_line(ax1, index, savgol_filter((data_noi_all_tws_mean),19,2), savgol_filter((data_noi_all_tws[1,:]),19,2), savgol_filter((data_noi_all_tws[0,:]),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((data_irr_all_tws_MED[2,:]),19,2)/365/86400, savgol_filter((data_irr_all_tws_MED[1,:])/365/86400,19,2), savgol_filter((data_irr_all_tws_MED[0,:])/365/86400,19,2), r'Mediterranean', '', 'tranirr', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line(ax1, index, savgol_filter((data_noi_all_tws_MED[2,:]),19,2)/365/86400, savgol_filter((data_noi_all_tws_MED[1,:])/365/86400,19,2), savgol_filter((data_noi_all_tws_MED[0,:])/365/86400,19,2), r'Mediterranean', '', '1901irr', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_no_grd(ax1, index, savgol_filter((data_irr_all_tws_MED[5,:])/365/86400,19,2), r'Mediterranean', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((data_noi_all_tws_MED[5,:])/365/86400,19,2), r'Mediterranean', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

x1 = np.array(range(1901, 1961))
param_irr_1_plot = param_irr_MED1
param_noi_1_plot = param_noi_MED1

y1_irr = param_irr_1_plot[0] * x1 + param_irr_1_plot[1]
y1_noi = param_noi_1_plot[0] * x1 + param_noi_1_plot[1]

x2 = np.array(range(1961, 2015))
param_irr_2_plot = param_irr_MED2
param_noi_2_plot = param_noi_MED2

y2_irr = param_irr_2_plot[0] * x2 + param_irr_2_plot[1]
y2_noi = param_noi_2_plot[0] * x2 + param_noi_2_plot[1]

plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[5], y1_irr[30]-900, r'-3.962***($\pm$0.181)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[5], y1_noi[10]+800, '-3.099***($\pm$0.080)', fontsize = 14, color = 'brown', weight='bold')
plt.legend(loc = 'upper left', fontsize = 14)
plt.plot(x2, y2_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x2[0], y2_irr[10]-1200, r'-11.000***($\pm$0.286)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x2, y2_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x2[0], y2_noi[10]+600, '-4.177***($\pm$0.059)', fontsize = 14, color = 'brown', weight='bold')
# plot_line_IPSL(ax1, index, savgol_filter((data_irr_all_tws_MED[3,:])/365/86400,19,2), r'Mediterranean', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
# plot_line_IPSL(ax1, index, savgol_filter((data_noi_all_tws_MED[3,:])/365/86400,19,2), r'Mediterranean', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

# plot_line_E3SM(ax1, index, savgol_filter((data_irr_all_tws_MED[4,:])/365/86400,19,2), r'Mediterranean', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
# plot_line_E3SM(ax1, index, savgol_filter((data_noi_all_tws_MED[4,:])/365/86400,19,2), r'Mediterranean', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([-5000, 5000])

ax1 = plt.subplot(424, frameon=True)
index = 'd'

x1 = np.array(range(1901, 1961))

param_diff_1_plot = param_diff_MED1


y1_diff = param_diff_1_plot[0] * x1 + param_diff_1_plot[1]


x2 = np.array(range(1961, 2015))

param_diff_2_plot = param_diff_MED2


y2_diff = param_diff_2_plot[0] * x2 + param_diff_2_plot[1]

plot_line(ax1, index, savgol_filter((data_diff_all_tws_MED[2,:])/365/86400,19,2), savgol_filter((data_diff_all_tws_MED[1,:])/365/86400,19,2), savgol_filter((data_diff_all_tws_MED[0,:])/365/86400,19,2), r'Mediterranean', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((data_diff_all_tws_MED[5,:])/365/86400,19,2), r'Mediterranean', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plot_line_IPSL(ax1, index, savgol_filter((data_diff_all_tws_MED[3,:])/365/86400,19,2), r'Mediterranean', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plot_line_E3SM(ax1, index, savgol_filter((data_diff_all_tws_MED[4,:])/365/86400,19,2), r'Mediterranean', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plt.plot(x1, y1_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.legend(loc = 'best', fontsize = 14)
plt.plot(x2, y2_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.text(x1[10], y1_diff[30]-500, r'-0.863***($\pm$0.111)', fontsize = 14, color = 'green', weight='bold')
plt.text(x2[0], y2_diff[50]-1000, r'-6.823***($\pm$0.316)', fontsize = 14, color = 'green', weight='bold')
plt.grid(linestyle = '--')
plt.ylim([-2000, 2000])





ax1 = plt.subplot(425, frameon=True)
index = 'e'
# plot_line(ax1, index, savgol_filter((data_noi_all_tws_mean),19,2), savgol_filter((data_noi_all_tws[1,:]),19,2), savgol_filter((data_noi_all_tws[0,:]),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((data_irr_all_tws_CNA[2,:])/365/86400,19,2), savgol_filter((data_irr_all_tws_CNA[1,:])/365/86400,19,2), savgol_filter((data_irr_all_tws_CNA[0,:])/365/86400,19,2), r'Central North America', '', 'tranirr', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line(ax1, index, savgol_filter((data_noi_all_tws_CNA[2,:])/365/86400,19,2), savgol_filter((data_noi_all_tws_CNA[1,:])/365/86400,19,2), savgol_filter((data_noi_all_tws_CNA[0,:])/365/86400,19,2), r'Central North America', '', '1901irr', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_no_grd(ax1, index, savgol_filter((data_irr_all_tws_CNA[5,:])/365/86400,19,2), r'Central North America', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((data_noi_all_tws_CNA[5,:])/365/86400,19,2), r'Central North America', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

# plot_line_IPSL(ax1, index, savgol_filter((data_irr_all_tws_CNA[3,:])/365/86400,19,2), r'Central North America', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
# plot_line_IPSL(ax1, index, savgol_filter((data_noi_all_tws_CNA[3,:])/365/86400,19,2), r'Central North America', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

# plot_line_E3SM(ax1, index, savgol_filter((data_irr_all_tws_CNA[4,:])/365/86400,19,2), r'Central North America', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
# plot_line_E3SM(ax1, index, savgol_filter((data_noi_all_tws_CNA[4,:])/365/86400,19,2), r'Central North America', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')



x1 = np.array(range(1901, 1921))
param_irr_1_plot = param_irr_CNA1
param_noi_1_plot = param_noi_CNA1

y1_irr = param_irr_1_plot[0] * x1 + param_irr_1_plot[1]
y1_noi = param_noi_1_plot[0] * x1 + param_noi_1_plot[1]

x2 = np.array(range(1921, 2015))

param_irr_2_plot = param_irr_CNA2
param_noi_2_plot = param_noi_CNA2

y2_irr = param_irr_2_plot[0] * x2 + param_irr_2_plot[1]
y2_noi = param_noi_2_plot[0] * x2 + param_noi_2_plot[1]

plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[5], y1_irr[10]-400, r'+1.932***($\pm$0.728)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[5], y1_noi[10]+400, '+0.441***($\pm$0.115)', fontsize = 14, color = 'brown', weight='bold')
plt.legend(loc = 'upper left', fontsize = 14)
plt.plot(x2, y2_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x2[40], y2_irr[10]-800, r'-6.883***($\pm$0.126)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x2, y2_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x2[40], y2_noi[10]+300, '-0.767***($\pm$0.114)', fontsize = 14, color = 'brown', weight='bold')

plt.grid(linestyle = '--')
plt.ylim([-2000, 2000])

ax1 = plt.subplot(426, frameon=True)
index = 'f'
x1 = np.array(range(1901, 1921))
param_diff_1_plot = param_diff_CNA1


y1_diff = param_diff_1_plot[0] * x1 + param_diff_1_plot[1]
x2 = np.array(range(1921, 2015))

param_diff_2_plot = param_diff_CNA2


y2_diff = param_diff_2_plot[0] * x2 + param_diff_2_plot[1]

plot_line(ax1, index, savgol_filter((data_diff_all_tws_CNA[2,:])/365/86400,19,2), savgol_filter((data_diff_all_tws_CNA[1,:])/365/86400,19,2), savgol_filter((data_diff_all_tws_CNA[0,:])/365/86400,19,2), r'Central North America', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((data_diff_all_tws_CNA[5,:])/365/86400,19,2), r'Central North America', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plot_line_IPSL(ax1, index, savgol_filter((data_diff_all_tws_CNA[3,:])/365/86400,19,2), r'Central North America', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plot_line_E3SM(ax1, index, savgol_filter((data_diff_all_tws_CNA[4,:])/365/86400,19,2), r'Central North America', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plt.plot(x1, y1_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.legend(loc = 'best', fontsize = 14)
plt.plot(x2, y2_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)

plt.text(x1[1], y1_diff[10]+200, r'+1.491***($\pm$0.843)', fontsize = 14, color = 'green', weight='bold')
plt.text(x2[30], y2_diff[70]-600, r'-6.116***($\pm$0.135)', fontsize = 14, color = 'green', weight='bold')
plt.grid(linestyle = '--')
plt.ylim([-1500, 1500])



ax1 = plt.subplot(427, frameon=True)
index = 'g'
# plot_line(ax1, index, savgol_filter((data_noi_all_tws_mean),19,2), savgol_filter((data_noi_all_tws[1,:]),19,2), savgol_filter((data_noi_all_tws[0,:]),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((data_irr_all_tws_WCA[2,:])/365/86400,19,2), savgol_filter((data_irr_all_tws_WCA[1,:])/365/86400,19,2), savgol_filter((data_irr_all_tws_WCA[0,:])/365/86400,19,2), r'West Central Asia', '', 'tranirr', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line(ax1, index, savgol_filter((data_noi_all_tws_WCA[2,:])/365/86400,19,2), savgol_filter((data_noi_all_tws_WCA[1,:])/365/86400,19,2), savgol_filter((data_noi_all_tws_WCA[0,:])/365/86400,19,2), r'West Central Asia', '', '1901irr', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_no_grd(ax1, index, savgol_filter((data_irr_all_tws_WCA[5,:])/365/86400,19,2), r'West Central Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((data_noi_all_tws_WCA[5,:])/365/86400,19,2), r'West Central Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

# plot_line_IPSL(ax1, index, savgol_filter((data_irr_all_tws_WCA[3,:])/365/86400,19,2), r'West Central Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
# plot_line_IPSL(ax1, index, savgol_filter((data_noi_all_tws_WCA[3,:])/365/86400,19,2), r'West Central Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

# plot_line_E3SM(ax1, index, savgol_filter((data_irr_all_tws_WCA[4,:])/365/86400,19,2), r'West Central Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
# plot_line_E3SM(ax1, index, savgol_filter((data_noi_all_tws_WCA[4,:])/365/86400,19,2), r'West Central Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

x1 = np.array(range(1901, 1961))
param_irr_1_plot = param_irr_WCA1
param_noi_1_plot = param_noi_WCA1

y1_irr = param_irr_1_plot[0] * x1 + param_irr_1_plot[1]
y1_noi = param_noi_1_plot[0] * x1 + param_noi_1_plot[1]

x2 = np.array(range(1961, 2014))
param_irr_2_plot = param_irr_WCA2
param_noi_2_plot = param_noi_WCA2

y2_irr = param_irr_2_plot[0] * x2 + param_irr_2_plot[1]
y2_noi = param_noi_2_plot[0] * x2 + param_noi_2_plot[1]

plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[5], y1_irr[30]-900, r'-9.175***($\pm$0.258)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[5], y1_noi[10]+800, '-7.342***($\pm$0.204)', fontsize = 14, color = 'brown', weight='bold')
plt.legend(loc = 'upper left', fontsize = 14)
plt.plot(x2, y2_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x2[0], y2_irr[10]-1300, r'-14.592***($\pm$0.229)', fontsize = 14, color = 'dodgerblue', weight='bold')
plt.plot(x2, y2_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x2[0], y2_noi[10]+800, '-8.420***($\pm$0.131)', fontsize = 14, color = 'brown', weight='bold')

plt.grid(linestyle = '--')
plt.ylim([-7000, 7000])

ax1 = plt.subplot(428, frameon=True)
index = 'h'

x1 = np.array(range(1901, 1961))
param_diff_1_plot = param_diff_WCA1


y1_diff = param_diff_1_plot[0] * x1 + param_diff_1_plot[1]

x2 = np.array(range(1961, 2015))
param_diff_2_plot = param_diff_WCA2


y2_diff = param_diff_2_plot[0] * x2 + param_diff_2_plot[1]

plot_line(ax1, index, savgol_filter((data_diff_all_tws_WCA[2,:])/365/86400,19,2), savgol_filter((data_diff_all_tws_WCA[1,:])/365/86400,19,2), savgol_filter((data_diff_all_tws_WCA[0,:])/365/86400,19,2), r'West Central Asia', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((data_diff_all_tws_WCA[5,:])/365/86400,19,2), r'West Central Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plot_line_IPSL(ax1, index, savgol_filter((data_diff_all_tws_WCA[3,:])/365/86400,19,2), r'West Central Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plot_line_E3SM(ax1, index, savgol_filter((data_diff_all_tws_WCA[4,:])/365/86400,19,2), r'West Central Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plt.plot(x1, y1_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.legend(loc = 'best', fontsize = 14)
plt.plot(x2, y2_diff, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.text(x1[10], y1_diff[30]-500, r'-1.834***($\pm$0.090)', fontsize = 14, color = 'green', weight='bold')
plt.text(x2[0], y2_diff[50]-1000, r'-6.172***($\pm$0.131)', fontsize = 14, color = 'green', weight='bold')
plt.grid(linestyle = '--')
plt.ylim([-2000, 2000])

